## Application Package publication on Resource Manager

In [1]:

import sys
import yaml
import os
from helpers import Atom, post_atom
from lxml import etree
import datetime
import hashlib

In [2]:
%load_ext autoreload

In [3]:
application_package = 'vegetation-index.cwl'

In [4]:
with open(application_package, 'r') as stream:
    try:
        cwl = yaml.safe_load(stream)

        stream.seek(0)

        cwl_content = ''.join(stream.readlines())
    except yaml.YAMLError as exc:
        raise

In [5]:
print(cwl_content)

$graph:
- baseCommand: vegetation-index
  class: CommandLineTool
  hints:
    DockerRequirement:
      dockerPull: terradue/eoepca-vegetation-index:0.1
  id: clt
  inputs:
    inp1:
      inputBinding:
        position: 1
        prefix: --input-reference
      type: Directory
    inp2:
      inputBinding:
        position: 2
        prefix: --aoi
      type: string
  outputs:
    results:
      outputBinding:
        glob: .
      type: Any
  requirements:
    EnvVarRequirement:
      envDef:
        PATH: /opt/anaconda/envs/env_vi/bin:/opt/anaconda/envs/env_vi/bin:/opt/anaconda/envs/env_default/bin:/opt/anaconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
        PREFIX: /opt/anaconda/envs/env_vi
    ResourceRequirement: {}
- class: Workflow
  doc: Vegetation index processor
  id: vegetation-index
  inputs:
    aoi:
      doc: Area of interest in WKT
      label: Area of interest
      type: string
    input_reference:
      doc: EO product for vegetation index
 

In [6]:
for elem in cwl['$graph']:

    if elem['class'] == 'Workflow':
        workflow = elem

In [7]:
m = hashlib.md5()

m.update(cwl_content.encode('utf-8'))

identifier = m.hexdigest()
    

In [24]:
identifier

'e563cc5596b02a189277de6228a9e2af'

In [8]:
template = '''<?xml version="1.0"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <entry>
    <title type="text"></title>
    <summary type="html"></summary>
    <date xmlns="http://purl.org/dc/elements/1.1/"></date>
    <identifier xmlns="http://purl.org/dc/elements/1.1/"></identifier>
  </entry>
</feed>'''
    

In [9]:
atom_template = Atom(etree.fromstring(template))

atom_template.set_identifier(identifier)
atom_template.set_title_text(workflow['label'])
atom_template.set_summary_text(workflow['doc'])

atom_template.set_dcdate(datetime.datetime.now().strftime('%Y-%m-%dT%H:%M%SZ'))

offering = atom_template.create_offering(code='http://www.opengis.net/eoc/applicationContext/cwl',
                                         content=cwl_content)

atom_template.add_offering(offering)

In [20]:
rm_endpoint = 'https://catalog.terradue.com/eoepca-services/description'

In [21]:
username = ''
api_key = ''

In [22]:
r = post_atom(atom_template,
              rm_endpoint,
              username, 
              api_key)

print(r.status_code, r.reason)

404 Not Found


In [23]:

with open('vegetation-index.xml', 'w') as f:
    
    f.write(b'<?xml version="1.0" encoding="UTF-8"?>\n'.decode('utf-8'))
    f.write(etree.tostring(atom_template.root, 
                                        pretty_print=True).decode('utf-8'))